### 진행 순서
1. 데이터베이스 생성
2. 상품 리스트 테이블 생성
3. 주문 목록 추가
4. 총 구매 가격 산출

### 데이터베이스 생성

In [5]:
import sqlite3

# 쇼핑몰 데이터베이스
conn = sqlite3.connect("shop.db")

# 커서 생성
c = conn.cursor()

### csv 파일로 테이블 추가

In [6]:
import pandas as pd

product_list = pd.read_csv('product_list.csv', encoding='euc-kr')
print(product_list)

    ID   상품명      가격
0    1    모자   15000
1    2    코트  200000
2    3   티셔츠   20000
3    4  블라우스   55000
4    5   가디건   45000
5    6   청바지   50000
6    7    구두  150000
7    8    가방  170000
8   10    양말   30000
9   10    양말   30000
10  10    양말   30000
11  10    양말  130000


In [7]:
# 상품 리스트 테이블에 추가
product_list.to_sql('product_list', conn, if_exists='append', index=False)

12

### 튜플(리스트)로 테이블 추가

In [9]:
# 테이블 삭제
c.execute("drop table product_list")
conn.commit()

OperationalError: no such table: product_list

In [10]:
# 추가할 상품 리스트
product_list = [
    ['모자', 10000],
    ['가방', 12000],
    ['가방', 120001],
    ['가방', 120002],
    ['가방', 120003],
    ['바지', 20000],
]

# 테이블 생성
c.execute("create table if not exists product_list (id integer primary key autoincrement, 상품명 text, 가격 integer)")

# 레코드 추가
c.executemany("insert into product_list (상품명, 가격) values (?, ?)", product_list)
conn.commit()

### 주문 목록 추가